In [1]:
get_ipython().system('pip install pyspark')


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession\
        .builder\
        .appName("TP_BDM-Doumi_Boudis")     .config("spark.some.config.option", "some-value")     .getOrCreate()


In [3]:
dataset= spark.read.csv('ad.data',header=False,inferSchema=True)


In [4]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import col


In [20]:
print(dataset.dtypes)

+---+---+---+---+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----

In [5]:
from pyspark.sql.types import IntegerType

dataset = dataset.withColumn("_c0", col("_c0").cast(IntegerType()))
dataset = dataset.withColumn("_c1", col("_c1").cast(IntegerType()))
dataset = dataset.withColumn("_c2", col("_c2").cast(IntegerType()))
dataset = dataset.withColumn("_c3", col("_c3").cast(IntegerType()))

In [28]:
print(dataset.dtypes)

[('_c0', 'int'), ('_c1', 'int'), ('_c2', 'int'), ('_c3', 'int'), ('_c4', 'int'), ('_c5', 'int'), ('_c6', 'int'), ('_c7', 'int'), ('_c8', 'int'), ('_c9', 'int'), ('_c10', 'int'), ('_c11', 'int'), ('_c12', 'int'), ('_c13', 'int'), ('_c14', 'int'), ('_c15', 'int'), ('_c16', 'int'), ('_c17', 'int'), ('_c18', 'int'), ('_c19', 'int'), ('_c20', 'int'), ('_c21', 'int'), ('_c22', 'int'), ('_c23', 'int'), ('_c24', 'int'), ('_c25', 'int'), ('_c26', 'int'), ('_c27', 'int'), ('_c28', 'int'), ('_c29', 'int'), ('_c30', 'int'), ('_c31', 'int'), ('_c32', 'int'), ('_c33', 'int'), ('_c34', 'int'), ('_c35', 'int'), ('_c36', 'int'), ('_c37', 'int'), ('_c38', 'int'), ('_c39', 'int'), ('_c40', 'int'), ('_c41', 'int'), ('_c42', 'int'), ('_c43', 'int'), ('_c44', 'int'), ('_c45', 'int'), ('_c46', 'int'), ('_c47', 'int'), ('_c48', 'int'), ('_c49', 'int'), ('_c50', 'int'), ('_c51', 'int'), ('_c52', 'int'), ('_c53', 'int'), ('_c54', 'int'), ('_c55', 'int'), ('_c56', 'int'), ('_c57', 'int'), ('_c58', 'int'), ('_c59

In [6]:
dataset = dataset.fillna(0)


In [7]:
modele_couche1 = [1558,500,100,2]
modele_couche2 = [1558,100,50,2]
modele_couche3 =[1558,1000,500,100,2]
modele_couche4 =[1558, 800, 400,200,2]
trainer = MultilayerPerceptronClassifier()

In [8]:
string_columns= [item[0] for item in dataset.dtypes if item[1].startswith(
    'string')] #Ici on sais bien que seulement la derniere colonne est de type string
indexer = StringIndexer(inputCol=string_columns[0] , outputCol="label") 
dataset = indexer.fit(dataset).transform(dataset) 
numeric_columns = [item[0] for item in dataset.dtypes if item[1].startswith(
    'int')]

featuresCreator = VectorAssembler(
    inputCols=numeric_columns,
    outputCol='features')
adData = featuresCreator.transform(dataset).select(col("features"),col("label"))



In [9]:
paramGrid = ParamGridBuilder().addGrid(trainer.maxIter,[5,10,20])\
            .addGrid(trainer.layers,[modele_couche1,modele_couche2,modele_couche3,modele_couche4])\
            .build()

crossValidator = CrossValidator(estimator=trainer,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(),
                          numFolds=3)



In [10]:
model = crossValidator.fit(adData)


In [11]:
model.bestModel.getLayers()


[1558, 500, 100, 2]

In [12]:
model.bestModel.getMaxIter()


20

In [13]:
classifier = MultilayerPerceptronClassifier(labelCol='label',
                                            featuresCol='features',
                                            maxIter=20,
                                            layers=modele_couche1,
                                            blockSize=128,
                                            seed=1234)



In [14]:
bestModel = classifier.fit(adData)


In [15]:
predictions = bestModel.transform(adData)


In [16]:
evaluator = MulticlassClassificationEvaluator(labelCol="label")
evaluator.evaluate(predictions)

0.9056745432429296

In [17]:
import pandas as pd
import numpy as np

df = dataset.select("*").toPandas()       
df.head()

,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,...,_c1550,_c1551,_c1552,_c1553,_c1554,_c1555,_c1556,_c1557,_c1558,label
0,125,125,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,ad.,1.0
1,57,468,8,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,ad.,1.0
2,33,230,6,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,ad.,1.0
3,60,468,7,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,ad.,1.0
4,60,468,7,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,ad.,1.0


In [18]:
cor = df.corr()
cor_target = abs(cor["label"])
relevant_features = cor_target[cor_target>0.3]
relevant_features

_c1       0.493244
_c351     0.521399
_c968     0.441370
_c1047    0.383151
_c1058    0.302114
_c1089    0.310056
_c1108    0.375385
_c1118    0.375385
_c1143    0.405642
_c1153    0.405642
_c1154    0.380579
_c1198    0.380579
_c1218    0.380579
_c1229    0.370716
_c1243    0.578361
_c1327    0.302114
_c1344    0.436293
_c1373    0.310056
_c1394    0.377990
_c1399    0.509780
_c1422    0.380579
_c1423    0.302114
_c1424    0.380579
_c1435    0.429429
_c1455    0.437062
_c1483    0.447766
_c1513    0.302779
_c1532    0.335705
label     1.000000
Name: label, dtype: float64

In [19]:
for columnn in df :
    if ((columnn not in relevant_features) and (columnn != "_c1558")):
         df = df.drop(columnn,1) 
df
    

,_c1,_c351,_c968,_c1047,_c1058,_c1089,_c1108,_c1118,_c1143,_c1153,...,_c1422,_c1423,_c1424,_c1435,_c1455,_c1483,_c1513,_c1532,_c1558,label
0,125,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,1,0,0,ad.,1.0
1,468,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,ad.,1.0
2,230,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,ad.,1.0
3,468,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,ad.,1.0
4,468,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,ad.,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3274,94,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,nonad.,0.0
3275,140,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,nonad.,0.0
3276,120,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,nonad.,0.0
3277,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,nonad.,0.0


In [22]:
spark.conf.set("spark.sql.execution.arrow.enabled","true")
newDataset=spark.createDataFrame(df) 
newDataset.show()

+---+-----+-----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-----+
|_c1|_c351|_c968|_c1047|_c1058|_c1089|_c1108|_c1118|_c1143|_c1153|_c1154|_c1198|_c1218|_c1229|_c1243|_c1327|_c1344|_c1373|_c1394|_c1399|_c1422|_c1423|_c1424|_c1435|_c1455|_c1483|_c1513|_c1532|_c1558|label|
+---+-----+-----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-----+
|125|    0|    0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     1|     0|     0|     0|     0|     0|     0|     0|     0|     1|     1|     1|     0|     0|   ad.|  1.0|
|468|    0|    0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     1|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     

In [23]:

attributs=newDataset.schema.names[0:28]
typee = newDataset.schema.names[28]
#indexer = StringIndexer(inputCol=typee , outputCol="label") 
#newDataset = indexer.fit(newDataset).transform(newDataset) 
featuresCreator = VectorAssembler(
    inputCols=attributs,
    outputCol='features')
newAdData = featuresCreator.transform(newDataset).select(col("features"),col("label"))



In [24]:
new_modele_couche = [28,500,100,2]
classifier = MultilayerPerceptronClassifier(labelCol='label',
                                            featuresCol='features',
                                            maxIter=20,
                                            layers=new_modele_couche,
                                            blockSize=128,
                                            seed=1234)



In [25]:

bestModel_bestAttribut = classifier.fit(newAdData)


In [26]:
newPrediction = bestModel_bestAttribut.transform(newAdData)
evaluator = MulticlassClassificationEvaluator(labelCol="label")
evaluator.evaluate(newPrediction)

0.7952948355789131